<a href="https://colab.research.google.com/github/AmandinChyba/Project1-Fields-2022/blob/main/Deep_Learning/wobNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import haiku as hk
import jax.numpy as jnp
import numpy as np
import jax
import optax

import pandas as pd

In [47]:
def getData():
  data = np.load('sample_data/wob_piecwise_data.npz')
  return data['x'], data['y']

In [16]:
def trueSoln(p):
  return 0.5 + np.arctan(2*p[1]/(1-np.power(p[0],2)-np.power(p[1],2)))/np.pi

In [21]:
def net_fn(x):
  mlp = hk.Sequential([
      hk.Flatten(),
      hk.Linear(200), jax.nn.relu,
      hk.Linear(50), jax.nn.relu,
      hk.Linear(1),
  ])
  return mlp(x)

In [52]:
x,y = getData()
x = jnp.array(x)
y = jnp.array(y)
samples, dimensions = x.shape

In [65]:
lr = 1e-3
batch_size = 10
key = jax.random.PRNGKey(0)

network = hk.without_apply_rng(hk.transform(net_fn))
optimiser = optax.adam(lr)

In [53]:
params = network.init(key, x[0])

In [71]:
def loss(params, input_data, actual):
    preds = network.apply(params, input_data)
    return jnp.power(actual - preds, 2).mean()

In [70]:
def update(params, grads):
  return params - lr * grads